In [230]:
import utils

import re
from collections import Counter
import numpy as np
from collections import defaultdict
from itertools import combinations
from math import gcd

## Day 8: Resonant Collinearity

[#](https://adventofcode.com/2024/day/8) We have a map of antennas on a 2d grid. Each antenna is tuned to a specific frequency - represented as a letter or digit.

Antennas apply effects (antinodes) when:
* two antennas are aligned
* at points where one antenna is 2x and another is 1x away.
* antinodes can occur at points where an antenna already is

So if you have three aligned antennas, you get more antinodes. Ignore ones off the grid. How many unique locations on the grid contain an antinode?

In [371]:
sample_input: str = """
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
"""

puzzle_input = utils.get_input(8, splitlines=False)

In [372]:
def parse_input(input_str=sample_input, debug=False):
    """returns 2d grid"""
    # return [
    #     [0 if char == "." else char for char in row]
    #     for row in input_str.strip().splitlines()
    # ]
    return [[char for char in row] for row in input_str.strip().splitlines()]


grid = parse_input(sample_input, True)
grid

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]

This seems like a job for numpy... trying regular python first:

1. Calculate each line where antennas of the same frequency are aligned.
2. Calculate antinodes, store their position in a list

In [373]:
def get_antennas(grid=grid):
    """returns a dict of antenna freq: position"""
    antennas = defaultdict(list)
    for y, row in enumerate(grid):
        for x, char in enumerate(row):
            if char != ".":
                antennas[char].append((x, y))
    return antennas


antennas = get_antennas()
antennas

defaultdict(list,
            {'0': [(8, 1), (5, 2), (7, 3), (4, 4)],
             'A': [(6, 5), (8, 8), (9, 9)]})

For each list of antennas, figure out which ones are aligned. Since we only care about pairs of antennas we can use [itertools.combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations) to generate all possible antenna pairs:

In [374]:
def get_antinodes(
    p1: tuple[int, int], p2: tuple[int, int], grid=grid, repeats=False, debug=False
):
    """takes two points, returns False or antinodes"""
    forms_line = False
    X, Y = len(grid[0]), len(grid)  # grid size
    (x1, y1), (x2, y2) = p1, p2
    dx, dy = x2 - x1, y2 - y1

    if (y1 == y2) or (x1 == x2):  # up or down line
        forms_line = True
    else:
        forms_line = gcd(abs(dx), abs(dy)) > 0

    if forms_line:
        antinodes = {
            (x1 + dx, y1 + dy),
            (x1 - dx, y1 - dy),
            (x2 + dx, y2 + dy),
            (x2 - dx, y2 - dy),
        } - {p1, p2}
        if debug:
            print(f"    {antinodes=} for {p1=} {p2=}")

        # only return antinodes on grid
        return [pos for pos in antinodes if ((0 <= pos[0] < X) and (0 <= pos[1] < Y))]
    else:
        return False


antenna_pairs = (((4, 3), (8, 4)), ((4, 3), (5, 5)), ((8, 4), (5, 5)))
for p1, p2 in antenna_pairs:
    print(f"{p1=} {p2=}")
    ants = get_antinodes(p1, p2, grid, debug=True)

p1=(4, 3) p2=(8, 4)
    antinodes={(12, 5), (0, 2)} for p1=(4, 3) p2=(8, 4)
p1=(4, 3) p2=(5, 5)
    antinodes={(3, 1), (6, 7)} for p1=(4, 3) p2=(5, 5)
p1=(8, 4) p2=(5, 5)
    antinodes={(11, 3), (2, 6)} for p1=(8, 4) p2=(5, 5)


In [375]:
def solve(inp: str = sample_input, repeats=False, debug: bool = False):
    grid = parse_input(inp)
    antennas = get_antennas(grid)

    antinodes = []
    for freq, ants in antennas.items():
        antenna_pairs = tuple(combinations(ants, 2))

        for p1, p2 in antenna_pairs:
            if ant_list := get_antinodes(p1, p2, grid, repeats=repeats):
                if debug:
                    print(f"   {p1} {p2} forms line with {ant_list=}")
                antinodes += ant_list

    return {"ans": len(set(antinodes)), "antinodes": antinodes}


assert solve()["ans"] == 14
results = solve(puzzle_input, False)
print(f"Part 1: {results["ans"]}")

Part 1: 252


## Part 2

is just saying anitnodes can repeat. So updating the `get_antinodes` function to handle repeats. Everything else should be the same. 

This is another case where numpy would make this easier, but sticking with python... 

Checking if two points formed a line stumped me for quite a while, so some googling said use gcd

```py
forms_line = gcd(abs(dx), abs(dy)) > 0
```

This is a trivial part of the problem, but where I got stuck. [gcd] returns the largest positive integer which divides two numbers, if its > 0 than the line passes  points with integer coordinates. Earlier I was only checking for perfectly diagnol lines, which was not giving me enough lines.



In [376]:
def get_antinodes(
    p1: tuple[int, int], p2: tuple[int, int], grid=grid, repeats=False, debug=False
):
    """takes two points, returns False or antinodes"""
    forms_line = False
    X, Y = len(grid[0]), len(grid)  # grid size
    (x1, y1), (x2, y2) = p1, p2
    dx, dy = x2 - x1, y2 - y1

    if (y1 == y2) or (x1 == x2):  # up or down line
        forms_line = True
    else:
        forms_line = gcd(abs(dx), abs(dy)) > 0

    if forms_line:
        antinodes = set()

        if repeats:

            # Extend the line in both directions (positive and negative)
            for i in range(1, max(X, Y)):
                new_positions = [
                    (x1 + i * dx, y1 + i * dy),
                    (x1 - i * dx, y1 - i * dy),
                    (x2 + i * dx, y2 + i * dy),
                    (x2 - i * dx, y2 - i * dy),
                ]
                for pos in new_positions:
                    if 0 <= pos[0] < X and 0 <= pos[1] < Y:
                        antinodes.add(pos)

        else:
            # Single-step antinodes
            antinodes.update(
                {
                    (x1 + dx, y1 + dy),
                    (x1 - dx, y1 - dy),
                    (x2 + dx, y2 + dy),
                    (x2 - dx, y2 - dy),
                }
            )
        if not repeats:
            antinodes = antinodes - {p1, p2}
        if debug:
            print(f"    {antinodes=} for {p1=} {p2=}")

        # only return antinodes on grid
        return [pos for pos in antinodes if ((0 <= pos[0] < X) and (0 <= pos[1] < Y))]
    else:
        return False


assert solve(sample_input, debug=False, repeats=True)["ans"] == 34
solve(puzzle_input, debug=False, repeats=True)["ans"]

839